In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import torch

# Load the Excel file
excel_path = '/content/drive/MyDrive/BERT_Sentiment/Labeled_Data.xlsx'  # Path to your Excel file
df = pd.read_excel(excel_path)

# Ensure all reviews are strings
df['Review'] = df['Review'].astype(str).fillna('')

# Load the fine-tuned BERT model and tokenizer
model_dir = "/content/drive/MyDrive/BERT_Sentiment"  # Directory where your model files are located
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(
    model_dir,
    config=f"{model_dir}/config.json"  # Path to your config.json file
)

# Function to predict sentiment and score
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    confidence, prediction = torch.max(probabilities, dim=-1)
    return prediction.item(), confidence.item()

# Apply the prediction function to each review
df['Predicted_Sentiment'], df['Confidence_Score'] = zip(*df['Review'].map(predict_sentiment))

# Save the updated dataframe to a new Excel file
output_excel_path = '/content/drive/MyDrive/BERT_Sentiment/Data_with_Predictions.xlsx'  # Path for the output Excel file
df.to_excel(output_excel_path, index=False)

print(f"Predictions saved to {output_excel_path}")


Predictions saved to /content/drive/MyDrive/BERT_Sentiment/Data_with_Predictions.xlsx
